In [22]:
import pandas as pd
import recordlinkage

hospital_accounts = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_info.csv',
    index_col='Account_Num'
)
hospital_reimbursement = pd.read_csv(
    'https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv',
    index_col='Provider_Num'
)

In [48]:
indexer = recordlinkage.Index()

# Columns to match on from df_left
left_on = ["Facility Name", "Address", "City", "State"]

# Columns to match on from df_right
right_on = [
    "Provider Name", "Provider Street Address", "Provider City",
    "Provider State"
]
#indexer.full()
indexer.sortedneighbourhood(left_on='State', right_on='Provider State')
#indexer.block(left_on, right_on)

<Index>

In [49]:
candidates = indexer.index(hospital_accounts, hospital_reimbursement)
print(len(candidates))

998860


In [50]:
len(hospital_reimbursement)*len(hospital_accounts)

14399283

In [53]:
compare = recordlinkage.Compare()
compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
               'Provider Name',
               threshold=0.85,
               label='Hosp_Name')
compare.string('Address',
               'Provider Street Address',
               method='jarowinkler',
               threshold=0.85,
               label='Hosp_Address')
features = compare.compute(candidates, hospital_accounts,
                           hospital_reimbursement)

In [54]:
features

City  Hosp_Name  Hosp_Address
Account_Num Provider_Num                               
10605       537184           0        0.0           0.0
            803181           0        0.0           0.0
            450616           0        0.0           0.0
            854377           0        0.0           0.0
            560361           0        0.0           0.0
...                        ...        ...           ...
70226       815904           0        0.0           0.0
            746090           0        0.0           0.0
            193062           0        0.0           0.0
            834984           0        0.0           0.0
            365095           0        0.0           0.0

[998860 rows x 3 columns]

In [55]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      2289
2.0       461
1.0      8190
0.0    987920
dtype: int64

In [56]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'City':'Hosp_Address'].sum(axis=1)
potential_matches.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score
0,51216,268781,0,1.0,1.0,2.0
1,55272,556917,1,1.0,1.0,3.0
2,87807,854637,1,1.0,1.0,3.0
3,51151,783146,1,0.0,1.0,2.0
4,11740,260374,1,1.0,1.0,3.0


In [81]:
import pandas as pd
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
import nltk
from nltk.stem.wordnet import WordNetLemmatizer

lemma = WordNetLemmatizer()

df=pd.read_csv(r'D:\MY STUFF\rhym\data/concatenated_ta.csv',index_col='name')
df.reset_index(inplace=True)

In [122]:
text='''The Winnti grouping of activity is large and may actually be a number of linked groups rather than a single discrete entity. Kaspersky describe Winnti as: 'The Winnti group has been attacking companies in the online video game industry since 2009 and is currently still active. The groups objectives are stealing digital certificates signed by legitimate software vendors in addition to intellectual property theft, including the source code of online game projects. The majority of the victims are from South East Asia.'''

In [123]:
stop_words = stopwords.words("english")+list(punctuation)
words = [w.lower() for sent in nltk.sent_tokenize(str(text)) for w in nltk.word_tokenize(sent)] #applying tokenization
words = [w for w in words if w not in stop_words and w.isalnum()] # removing stopwords and only alphanumeric
words = [w for w in words if len(w)>1] #removing if length is less than 1
tokens = [lemma.lemmatize(w,'v')  for w in words]

In [124]:
scoring=dict()
for index,row in df.iterrows():
    initial=True
    for i in tokens:
        if i in row['keywords'].split(' '):
            if initial:
                scoring[row['name']]=1
                initial = False
            else:
                scoring[row['name']] = scoring[row['name']]+1
            

In [125]:
sorted(scoring.items(), key=lambda x: x[1], reverse=True)

[('apt41', 18),
 ('lazarus group', 15),
 ('apt28', 14),
 ('apt37', 13),
 ('menupass', 13),
 ('oilrig', 13),
 ('patchwork', 13),
 ('cobalt group', 12),
 ('leviathan', 12),
 ('magic hound', 12),
 ('winnti group', 12),
 ('apt19', 11),
 ('copykittens', 11),
 ('platinum', 11),
 ('threat group-3390', 11),
 ('apt29', 10),
 ('apt32', 10),
 ('apt38', 10),
 ('axiom', 10),
 ('elderwood', 10),
 ('ta505', 10),
 ('the white company', 10),
 ('apt33', 9),
 ('bronze butler', 9),
 ('darkhotel', 9),
 ('fin5', 9),
 ('fin6', 9),
 ('fin7', 9),
 ('fin8', 9),
 ('kimsuky', 9),
 ('molerats', 9),
 ('night dragon', 9),
 ('rancor', 9),
 ('temp.veles', 9),
 ('wirte', 9),
 ('apt3', 8),
 ('apt39', 8),
 ('dark caracal', 8),
 ('darkhydrus', 8),
 ('deep panda', 8),
 ('dragonfly 2.0', 8),
 ('fin10', 8),
 ('gorgon group', 8),
 ('leafminer', 8),
 ('machete', 8),
 ('muddywater', 8),
 ('rtm', 8),
 ('silverterrier', 8),
 ('suckfly', 8),
 ('apt1', 7),
 ('apt17', 7),
 ('apt18', 7),
 ('fin4', 7),
 ('gallmaker', 7),
 ('group5', 7